In [126]:
import joblib
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import numpy as np


In [127]:
data = pd.read_csv("properties.csv")

In [128]:
#Age of the building is calculated from the construction year
data['Age_of_building'] = 2024 - data['construction_year']

In [129]:
data.columns

Index(['id', 'price', 'property_type', 'subproperty_type', 'region',
       'province', 'locality', 'zip_code', 'latitude', 'longitude',
       'construction_year', 'total_area_sqm', 'surface_land_sqm',
       'nbr_frontages', 'nbr_bedrooms', 'equipped_kitchen', 'fl_furnished',
       'fl_open_fire', 'fl_terrace', 'terrace_sqm', 'fl_garden', 'garden_sqm',
       'fl_swimming_pool', 'fl_floodzone', 'state_building',
       'primary_energy_consumption_sqm', 'epc', 'heating_type',
       'fl_double_glazing', 'cadastral_income', 'Age_of_building'],
      dtype='object')

In [130]:
data.isnull().sum()

id                                    0
price                                 0
property_type                         0
subproperty_type                      0
region                                0
province                              0
locality                              0
zip_code                              0
latitude                          14098
longitude                         14098
construction_year                 33391
total_area_sqm                     7615
surface_land_sqm                  36256
nbr_frontages                     26346
nbr_bedrooms                          0
equipped_kitchen                      0
fl_furnished                          0
fl_open_fire                          0
fl_terrace                            0
terrace_sqm                       13140
fl_garden                             0
garden_sqm                         2939
fl_swimming_pool                      0
fl_floodzone                          0
state_building                        0


"zip_code" though categorical but dropped from dataframe

In [131]:
num_features = ["primary_energy_consumption_sqm","garden_sqm", "cadastral_income", "latitude", "longitude", "Age_of_building", "total_area_sqm", "surface_land_sqm", "nbr_frontages", "nbr_bedrooms", "terrace_sqm"  ]
fl_features = ["fl_terrace", "fl_floodzone", "fl_swimming_pool", "fl_floodzone","fl_double_glazing" ]
cat_features = ['property_type', 'subproperty_type', 'region','province', 'locality','state_building','epc', 'heating_type',]

In [132]:
X = data[num_features + fl_features + cat_features]
y = data["price"]

In [133]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=505)

In [134]:
# Impute missing values using SimpleImputer
imputer = SimpleImputer(strategy="mean")
imputer.fit(X_train[num_features])
X_train[num_features] = imputer.transform(X_train[num_features])
X_test[num_features] = imputer.transform(X_test[num_features])

In [135]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train_standardized = scaler_X.fit_transform(X_train[num_features])
X_test_standardized = scaler_X.transform(X_test[num_features])

y_train_standardized = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
y_test_standardized = scaler_y.fit_transform(y_test.values.reshape(-1, 1))

# Initialize Isolation Forest to identify outliers
clf = IsolationForest(contamination=0.1)  # Adjust contamination based on your dataset

# Fit Isolation Forest on training data and predict outliers
outliers_train = clf.fit_predict(X_train_standardized)
outliers_test = clf.fit_predict(X_test_standardized)

# Remove outliers from training data
X_train_clean = (X_train[num_features])[outliers_train != -1]
y_train_clean = y_train[outliers_train != -1]

X_test_clean = (X_test[num_features])[outliers_test != -1]
y_test_clean = y_test[outliers_test != -1]

In [136]:
scaler = StandardScaler()
    scaler.fit(X_train[num_features])
    X_train_standardized = scaler.transform(X_train[num_features])
    X_test_standardized = scaler.transform(X_test[num_features])

IndentationError: unexpected indent (1354054268.py, line 2)

In [ ]:
# Convert categorical columns with one-hot encoding using OneHotEncoder
enc = OneHotEncoder()
enc.fit(X_train[cat_features])
enc.fit(X_test[cat_features])
X_train_cat = enc.transform(X_train[cat_features]).toarray()
X_test_cat = enc.transform(X_test[cat_features]).toarray()

In [ ]:
X_Ntrain = pd.concat(
    [
        X_train_clean.reset_index(drop=True),
        X_train[fl_features].reset_index(drop=True),
        pd.DataFrame(X_train_cat, columns=enc.get_feature_names_out()),
    ],
    axis=1,
)

X_Ntest = pd.concat(
        [
            X_test_clean.reset_index(drop=True),
            X_test[fl_features].reset_index(drop=True),
            pd.DataFrame(X_test_cat, columns=enc.get_feature_names_out()),
        ],
        axis=1,
    )




In [ ]:
model = LinearRegression()
model.fit(X_Ntrain, y_train)

# Predict on test set
y_pred = model.predict(X_Ntest)

# Evaluate model performance
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

ValueError: could not convert string to float: 'APARTMENT'

No standardization for y
scaler_y = StandardScaler()
scaler_y.fit(y_train.reshape(-1, 1))
y_train_scaled = scaler_y.fit(y_train.reshape(-1, 1))
y_test_scaled = scaler_y.fit(y_test.reshape(-1,1))




In [ ]:
print(f"Features: \n {X_train.columns.tolist()}")

    # Train the model
model = LinearRegression()
model.fit(X_train, y_train)


Features: 
 ['primary_energy_consumption_sqm', 'garden_sqm', 'cadastral_income', 'latitude', 'longitude', 'Age_of_building', 'total_area_sqm', 'surface_land_sqm', 'nbr_frontages', 'nbr_bedrooms', 'terrace_sqm', 'fl_terrace', 'fl_floodzone', 'fl_floodzone', 'fl_swimming_pool', 'fl_floodzone', 'fl_floodzone', 'fl_double_glazing', 'property_type_APARTMENT', 'property_type_HOUSE', 'subproperty_type_APARTMENT', 'subproperty_type_APARTMENT_BLOCK', 'subproperty_type_BUNGALOW', 'subproperty_type_CASTLE', 'subproperty_type_CHALET', 'subproperty_type_COUNTRY_COTTAGE', 'subproperty_type_DUPLEX', 'subproperty_type_EXCEPTIONAL_PROPERTY', 'subproperty_type_FARMHOUSE', 'subproperty_type_FLAT_STUDIO', 'subproperty_type_GROUND_FLOOR', 'subproperty_type_HOUSE', 'subproperty_type_KOT', 'subproperty_type_LOFT', 'subproperty_type_MANOR_HOUSE', 'subproperty_type_MANSION', 'subproperty_type_MIXED_USE_BUILDING', 'subproperty_type_OTHER_PROPERTY', 'subproperty_type_PENTHOUSE', 'subproperty_type_SERVICE_FLAT', 

LinearRegression()

In [ ]:
# Evaluate the model
train_score = r2_score(y_train, model.predict(X_train))
test_score = r2_score(y_test, model.predict(X_test))
print(f"Train R² score: {train_score}")
print(f"Test R² score: {test_score}")

Train R² score: 0.3451525016184904
Test R² score: 0.38471901045816426


In [ ]:
numeric_transformer = Pipeline(steps=('imputer', SimpleImputer(strategy='mean')),('scaler', StandardScaler))

SyntaxError: positional argument follows keyword argument (2690872991.py, line 1)

In [ ]:
g=sns.heatmap(df,vmin=0,vmax=1)
plot.show(g)

ValueError: could not convert string to float: 'APARTMENT'

Variables prefixed with fl_ are dummy variables (1/0)
Variables suffixed with _sqm indicate the measurement is in square meters
All missing categories for the categorical variables are encoded as MISSING

 # Split the data into features and target

In [ ]:
X = df[['garden_sqm' , 'primary_energy_consumption_sqm', 'fl_garden']]
y = df["price"]

# Split the data into training and testing sets
RandomState instance that specifies the seed value for the random number generator.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.20, random_state=505
    )

# Impute missing values using SimpleImputer

In [ ]:
df.price.mean

<bound method Series.mean of 0        225000.0
1        449000.0
2        335000.0
3        501000.0
4        982700.0
           ...   
75506    210000.0
75507    780000.0
75508    798000.0
75509    575000.0
75510    515000.0
Name: price, Length: 75511, dtype: float64>

In [ ]:
# just to check
df.primary_energy_consumption_sqm.isnull()
df.primary_energy_consumption_sqm.isnull().sum()
df.primary_energy_consumption_sqm.mean()
df.primary_energy_consumption_sqm.describe()


count    4.894400e+04
mean     1.688748e+03
std      1.589309e+05
min     -1.400000e+02
25%      1.280000e+02
50%      2.420000e+02
75%      3.920000e+02
max      2.023112e+07
Name: primary_energy_consumption_sqm, dtype: float64

In [ ]:
imputer = SimpleImputer(strategy="mean")
imputer.fit(X_train[df.primary_energy_consumption_sqm])
X_train[df.primary_energy_consumption_sqm] = imputer.transform(X_train[df.primary_energy_consumption_sqm])
X_test[df.primary_energy_consumption_sqm] = imputer.transform(X_test[df.primary_energy_consumption_sqm])


KeyError: "None of [Index([231.0, 221.0,   nan,  99.0,  19.0,   nan,   nan,   nan, 212.0, 394.0,\n       ...\n        91.0,   nan, 369.0,   nan, 217.0,   nan,  95.0, 351.0, 269.0,   nan],\n      dtype='float64', length=75511)] are in the [columns]"

In [ ]:
enc = OneHotEncoder()
enc.fit(X_train[cat_features])
X_train_cat = enc.transform(X_train[cat_features]).toarray()
X_test_cat = enc.transform(X_test[cat_features]).toarray()

In [ ]:
X_train = pd.concat(
        [
            X_train[num_features + fl_features].reset_index(drop=True),
            pd.DataFrame(X_train_cat, columns=enc.get_feature_names_out()),
        ],
        axis=1,
    )

In [ ]:
X_test = pd.concat(
        [
            X_test[num_features + fl_features].reset_index(drop=True),
            pd.DataFrame(X_test_cat, columns=enc.get_feature_names_out()),
        ],
        axis=1,
    )


In [ ]:
print(f"Features: \n {X_train.columns.tolist()}")

Features: 
 ['garden_sqm', 'primary_energy_consumption_sqm', 'fl_garden_0', 'fl_garden_1']


In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values